In [1]:
import biogeo_cophylo_simstudy
import treeducken_tools as td
import os
import dendropy
import subprocess
from dendropy import Tree
import pandas as pd
import re

nt = 12
turnover = 0.5
br = 0.07
dr = 0.035
trr = 0.5
gbr = 0.0
gdr = 0.0
reps = 1
nl = 1
ng = 1
ipp = 1
ne = 1
sd1 = 1859
sd2 = 2019


settings_regime_name = "test"
settings_dictionary = td.create_settings_dictionary(sbr=br, sdr=dr, lgtr=trr, gbr=gbr, gdr=gdr, num_loci=nl,
                                                        reps=reps, num_genes=1, ipp=1, ne=1, ntax=nt, screen_out=0,
                                                        ofn=settings_regime_name, sd1=sd1, sd2=sd2)

settings = td.write_settings_file(settings_dictionary, settings_regime_name)


subprocess.call("../src/create_directories_for_simdata.sh", shell=True)

0

In [2]:
run_cmd = "treeducken" + " -i " + "biogeo_cophylo_simstudy/settings/" + "test_settings.txt"
so = os.popen(run_cmd).read()
print(so)
setting_prefix = "test"
mv_cmd = "mv " + setting_prefix + "*" + " ./data/test"
so2 = os.popen(mv_cmd).read()
print(so2)


############################################################
####	treeducken, version 0.1 			####
####	6b534100b3acc4e8073baadb9fbf5694854134ca	####
############################################################




In [3]:
host_tree_suffix_fn = ".sp.tre"
symb_tree_suffix_fn = "_0.loc.tre"

brtimes_suffix_fn = ".times.txt"
ranges_suffix_fn = ".range.nex"

data_dir = "data/test/"

prefix_fn = "test_"


# read in the host and symbiont trees
host_fn = data_dir + prefix_fn + str(0) + host_tree_suffix_fn
symb_fn = data_dir + prefix_fn + str(0) + symb_tree_suffix_fn

host_tree = Tree.get(path=host_fn, schema="nexus", rooting="default-rooted")
symb_tree = Tree.get(path=symb_fn, schema="nexus", rooting="default-rooted")

# prune extinct tips on symb tree
symb_newick_tree = symb_tree.as_string("nexus")
symb_tree = dendropy.Tree.get_from_string(symb_newick_tree, "nexus")
ext_labels = re.findall("X\d+.{1}\d+", symb_newick_tree)
taxa_set = re.findall("T\d+.{1}\d+", symb_newick_tree)
for k in range(0, len(ext_labels)):
    ext_labels[k] = re.sub("_", " ", ext_labels[k])
for m in range(0, len(taxa_set)):
    taxa_set[m] = re.sub("_", " ", taxa_set[m])
symb_tree.prune_taxa_with_labels(ext_labels, update_bipartitions=True)
symb_tree.write(path=data_dir + prefix_fn + str(0) + "_pruned" + symb_tree_suffix_fn,
                        schema="nexus",
                        suppress_taxa_blocks=True)

# print out epoch times (no uncertainty in age here)
out_fn = data_dir + prefix_fn + str(0)
brtimes_out_fn = out_fn + brtimes_suffix_fn
biogeo_cophylo_simstudy.print_branching_times(host_tree, brtimes_out_fn)

# print out range files
nex_range_outfn = out_fn + ranges_suffix_fn
biogeo_cophylo_simstudy.print_nexus_range_file(nex_range_outfn, host_tree, symb_tree)

# print out connectivity graphs
connectivity_prefix_outfn = out_fn + ".connectivity"
biogeo_cophylo_simstudy.print_connectivity_graph(host_tree=host_tree, of_prefix=connectivity_prefix_outfn)

# print out "distance" matrix
dist_mat = host_tree.phylogenetic_distance_matrix()
distmat_ofn = out_fn + ".distances.txt"
dm_df = pd.DataFrame(data=dist_mat, index=host_tree.taxon_namespace)
biogeo_cophylo_simstudy.print_rev_script(data_dir, prefix_fn, 0)